In [1]:
from pymongo import MongoClient
import matplotlib.pyplot as plt

# Connect to your MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['vaers']  
collection = db['reports']  

In [3]:
# Aggregation pipeline with allowDiskUse option
pipeline = [
    {
        "$project": {
            "vax_data": 1,
            "symptoms": 1,
            "_id": 0
        }
    },
    {
        "$unwind": "$symptoms"
    },
    {
        "$group": {
            "_id": {
                "vaccine": "$vax_data.VAX_TYPE",
                "manufacturer": "$vax_data.VAX_MANU",
                "Event": "$symptoms"
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "vaccine": "$_id.vaccine",
            "manufacturer": "$_id.manufacturer",
            "Event": "$_id.Event"
        }
    }
]

# Execute aggregation pipeline with allowDiskUse option
results = collection.aggregate(pipeline, allowDiskUse=True)

# Print unique combinations
for result in results:
    print(result)

{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Abdominal discomfort'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Acute lymphocytic leukaemia'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Agitation'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Alanine aminotransferase increased'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Allergy test positive'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Allergy to metals'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Alpha 1 foetoprotein'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Anaphylactic reaction'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITHKLINE BIOLOGICALS'], 'Event': 'Anger'}
{'vaccine': ['6VAX-F'], 'manufacturer': ['GLAXOSMITH

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood testosterone normal'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood thyroid stimulating hormone'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood thyroid stimulating hormone abnormal'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood thyroid stimulating hormone decreased'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood thyroid stimulating hormone increased'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood thyroid stimulating hormone normal'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood triglycerides decreased'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood urea'}
{'vaccine': ['VARZOS'], 'manufacturer': ['MERCK & CO. INC.'], 'Event': 'Blood urea increased'}
{'vaccine': ['VARZOS'], 'manufacturer': 

In [13]:
# Aggregation pipeline with allowDiskUse option
pipeline = [
    {
        "$project": {
            "vax_data": 1,
            "symptoms": 1,
            "_id": 0
        }
    },
    {
        "$unwind": "$vax_data"
    },
    {
        "$match": {
            "vax_data.VAX_TYPE": {"$regex": "COVID"}
        }
    },
    {
        "$unwind": "$symptoms"
    },
    {
        "$group": {
            "_id": "$vax_data.VAX_TYPE"
        }
    }
]

# Execute aggregation pipeline with allowDiskUse option
results = collection.aggregate(pipeline, allowDiskUse=True)

# Extract unique vaccine values
vaccine_values = [result['_id'] for result in results]

# Print unique vaccine values
print("Distinct vaccine values:")
for value in vaccine_values:
    print(value)

In [30]:
# Aggregation pipeline with allowDiskUse option
pipeline = [
    {
        "$project": {
            "vax_data": 1,
            "symptoms": 1,
            "_id": 0
        }
    },
    {
        "$unwind": "$vax_data"
    },
    {
        "$match": {
            "vax_data.VAX_TYPE": {"$regex": "COVID"}
        }
    },
    {
        "$unwind": "$symptoms"
    },
    {
        "$group": {
            "_id": {
                "vaccine": "$vax_data.VAX_TYPE",
                "manufacturer": "$vax_data.VAX_MANU",
                "Event": "$symptoms"
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "vaccine": "$_id.vaccine",
            "manufacturer": "$_id.manufacturer",
            "Event": "$_id.Event",
            "count": "$count"
        }
    }
]

# Execute aggregation pipeline with allowDiskUse option
results = collection.aggregate(pipeline, allowDiskUse=True)




IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [32]:
# Create a list of unique combinations and their counts
unique_combinations = []
for result in results:
    unique_combinations.append(result)

# Print the list
print(unique_combinations)

[]


SyntaxError: invalid syntax (2394716835.py, line 1)